## AI4PEX SINDBAD Tutorial 
## 1. Parameter inversion exercise
A notebook by Sujan Koirala, Xu Shan, Jialiang Zhou and Nuno Carvalhais

---
## SINDBAD
[SINDBAD](http://sindbad-mdi.org/) is a model-data integration framework for terrestrial carbon-water processes [[Koirala et al., in prep.](https://essopenarchive.org/users/551954/articles/1271244)]. Is built in Julia with a view on speed and differenciability for the development of representation of processes and responses of ecosystem functioning to meteorological conditions and changes in climate. Sets on the concept of modularity to formaly test hypothesis on the representation of processes / models ($f(X,\theta)$), for given observational constraints ($Y$) and drivers ($X$) of the carbon and water dynamics in terrestrial ecosystems. Modularity is extended to cost functions ($\mathcal{L(\theta)}$) and optimization algorithms ($\mathcal{O}$). SINDBAD integrates machine learning for enhancing the representation of processes in mechanistically-inspired models, hybrid modeling [Reichstein et al., 2019], by learning ML-based parameterizations [e.g. Bao et al., 2024], paving way for process abstraction [Son et al., 2024].

---
## WROASTED: a Simple Coupled Carbon–Water Ecosystem Model
The carbon dynamics,  $\frac{dC}{dt}$, are simulated as the difference between gross assimilation and respiratory fluxes
$$
\frac{dC}{dt} = GPP - R_{ECO}
$$

where ${GPP}$, gross primary productivity, results from photosynthetic activity and $R_{ECO}$, ecosystem respiration, is the sum of autotrophic and heterotrophic respiratory fluxes, namely, $R_{A}$ and $R_{H}$. 

$R_{A}$ integrates both maintenance and growth respiration, $R_{M}$ and $R_{G}$, where  $R_{M}$  can be generically written like:

$$
R_{M} =\sum_{i=1}^{N} \tau_i \cdot C_i \cdot f_T 
$$

$i$ representing the different carbon pools ($C_i$) in vegetation - root/wood/leaf/reserves; $\tau_i$ the turnover rate of pool $i$ , and $f_T$ the temperature dependence of metabolic activity, usually a $Q_{10}$ function; while $R_G=Y_G \cdot GPP$, being $Y_G$ and constant growth efficiency parameter [see Amthor, 2001]. 

$R_H$ results from litter and soil decomposition:
$$
R_{H} =\sum_{i=1}^{N} \tau_i \cdot C_i \cdot f_T \cdot f_W
$$

$i$ representing the different heterotrophic carbon pools ($C_i$) in soils - fast and slow litter and organic carbon pools; $\tau_i$ the turnover rate of pool $i$ , $f_T$ and $f_W$ the temperature and soil moisture sensitivity of decomposition function.

Soil moisture dynamics, $\frac{dW}{dt}$:
$$
\frac{dW}{dt} = P_r - E_i - E_s - Q - D - T_r
$$

Being: $Pr$: precipitation; $E_i$: interception evaporation; $E_s$: soil evaporation; $Q$: surface runoff; $D$: drainage; $T_r$: plant transpiration.

Transpiration is tighly coupled to $GPP$, estimated as: 

$$
GPP = min(GPP_D,GPP_S)
$$

Being demand $GPP$:
$$
GPP_S = \epsilon^* \cdot f\text{APAR} \cdot \text{PAR} \cdot (f_L \cdot f_{CI} \cdot f_T \cdot f_{VPD} \cdot f_W)
$$
The product between: maximum light use efficiency, $\epsilon^*$; the fraction of photosynthetically active radiation, $\text{APAR}$, absorbed by leafs, $f\text{APAR}$; and the instantaneous effect of light intensity $f_L$, cloudiness index $f_CI$, vapor pressure deficit $f_VPD$ and soil moisture $f_W$ [see Bao et al., 2023; 2024].

And  supply $GPP$:

$$
GPP_S = PAW^{k_{Tr}} \cdot WUE
$$

Where where the daily variations in water use efficiency, $WUE$, result from changes in $VPD$ and $\quad [CO_2]_{atm}$. Upon $C$ assimilation by vegetation, and deduced $R_A$ costs, the available carbon is transported to the different vegetation pools depending on environmental conditions, as inspired by the growind season index (GSI) model [see Koirala et al., in print; Jolly et al., 2005]. 

Overall, WROASTED includes >40 parameters controlling the responses of carbon and water dynamics in terrestrial ecosystems constrainable by observations of ecosystem fluxes, eddy covariance, plant phenology from remote sensing EO data, and above ground biomass stocks, where available [see Koirala et al., in print].

---
## The challenge
To calibrate and generalize the model parameterization.

---
## Parameter inversion
The goal is to find $\theta$ such that the model predictions $f(X, \theta)$ best match observed datasets $y$. Here, the terrestrial ecosystem model, WROASTED, represented by $f(X, \theta)$, predicts a set of ecosystem carbon and water state and flux variables, $\hat{y}$, observed at locations: 
- $X$: meteorological drivers (i.e., temperature, radiation, precipitation, $VPD$, etc);
- $\theta$: parameter vector to be estimated;
- $y$: observations (e.g., $GPP$, $T_r$, evapotranspiration, $R_{ECO}$, aboveground biomass AGB, $f\text{APAR}$)

### Optimization problem
Generically can be written:
$$
\theta^*=\arg\min_{\theta \in \Theta} \; \mathcal{L}(\theta)\quad\text{via}\quad\mathcal{O}
$$

Where:
- $\mathcal{L}(\theta)$: is the cost function quantifying the mismatch between model predictions and observations,
- $\Theta$: feasible parameter space (e.g., bounds or priors on $\theta$),
- $\mathcal{O}$: optimization operator/algorithm (e.g., gradient descent, L-BFGS, CMA-ES)

In the exercise here, for fluxes and phenology time series, the loss function $\mathcal{L}(\theta)$ is set to the normalized Nash-Sutcliffe Efficiency (NNSE)
$$
\text{NNSE}(\theta) = 1 - \frac{1}{2-NSE}
$$
$$
\text{NSE}(\theta) = 1 - \frac{\sum_{i=1}^{N} (y_i - f(X_i, \theta))^2}{\sum_{i=1}^{N} (y_i - \bar{y})^2}
$$

While for stocks, AGB, an adjusted normalized mean average error is used
$$
NMAE = \frac{\sum_{i=1}^{N} |y_i - f(X_i, \theta)|}{N \cdot (1+ \bar{y})}
$$
$$
\theta^* = \arg\min_{\theta} \; \mathcal{L}(\theta)
$$

---
## Setting up SINDBAD-Tutorials
Navigate to the [SINDBAD-Tutorials for AI4PEX repository](GitHubLink) and install. Please follow instructions. For us, [VS Code](https://code.visualstudio.com/) has been a very fluid host for [Julia](https://julialang.org/) developments.

### Get the data for these SINDBAD tutorials
The data can be found [here](https://nextcloud.bgc-jena.mpg.de/s/w2mbH59W4nF3Tcd). Suggestion, store it in a child folder of the SINDBAD-Tutorials (e.g. SINDBAD-Tutorials/data/).

## Let's go...
### Get packages and goodies to go.

In [12]:
# cd("tutorials/ai4pex_2025/")
if Sys.iswindows()
    ENV["USER"] = Sys.iswindows() ? ENV["USERNAME"] : ENV["USER"]
end

# ================================== using tools ==================================================
# some of the things that will be using... Julia tools, SINDBAD tools, local codes...
using Revise
using SindbadTutorials
using SindbadTutorials.Dates
using SindbadTutorials.Plots
using SindbadTutorials.SindbadVisuals
toggleStackTraceNT()
include("tutorial_helpers.jl")

getSpinupSequenceSite (generic function with 2 methods)

### Get the data and paths to data setup

In [13]:
# ================================== get data / set paths ========================================= 
# data to be used can be found here: https://nextcloud.bgc-jena.mpg.de/s/w2mbH59W4nF3Tcd
# organizing the paths of data sources and outputs for this experiment
path_input_dir      = getSindbadDataDepot(; env_data_depot_var="SINDBAD_DATA_DEPOT", 
                    local_data_depot=joinpath(@__DIR__,"..","..","data","ai4pex_2025")); # for convenience, the data file is set within the SINDBAD-Tutorials path; this needs to be changed otherwise.
path_input          = joinpath("$(path_input_dir)","FLUXNET_v2023_12_1D_REPLACED_Noise003_v1.zarr"); # zarr data source containing all the data necessary for the exercise
path_observation    = path_input; # observations (synthetic or otherwise) are included in the same file
path_output         = "";


### That Zarr file contains many eddy covariance sites - synthetic data. Let's select one to invert.

In [14]:
# ================================== selecting a site =============================================
# there is a collection of several sites in the data files site info; #68 is DE-Hai
site_index      = 68;
domain, y_dist  = getSiteInfo(site_index);

### Now, setting up the experiment.
Here, the full experiment is set up. Includes links to JSON files with...

In [15]:
# ================================== setting up the experiment ====================================
# experiment is all set up according to a (collection of) json file(s)
experiment_json     = joinpath(@__DIR__,"settings_WROASTED_HB","experiment_insitu.json");
experiment_name     = "WROASTED_inversion_CMAES";
begin_year          = 1979;
end_year            = 2017;
run_optimization    = true;
isfile(experiment_json) ? nothing : println("Hmmm... does not exist : $(experiment_json)");

# setting up the model spinup sequence : can change according to the site...
spinup_sequence = getSpinupSequenceSite(y_dist, begin_year);

# default setting in experiment_json will be replaced by the "replace_info"
replace_info = Dict("experiment.basics.time.date_begin" => "$(begin_year)-01-01",
    "experiment.basics.domain" => domain,
    "experiment.basics.name" => experiment_name,
    "experiment.basics.time.date_end" => "$(end_year)-12-31",
    "experiment.flags.run_optimization" => run_optimization,
    "experiment.model_spinup.sequence" => spinup_sequence,
    "forcing.default_forcing.data_path" => path_input,
    "forcing.subset.site" => [site_index],
    "experiment.model_output.path" => path_output,
    "optimization.observations.default_observation.data_path" => path_observation,
    );



In [17]:
# ================================== forward run ================================================== 
# before running the optimization, check a forward run 
@time out_dflt  = runExperimentForward(experiment_json; replace_info=deepcopy(replace_info)); # full default model

# access some of the internals to do some plots with the forward runs...
info            = getExperimentInfo(experiment_json; replace_info=deepcopy(replace_info)); # note that this will modify information from json with the replace_info
forcing         = getForcing(info); 
run_helpers     = prepTEM(forcing, info); # not needed now
observations    = getObservation(info, forcing.helpers);
obs_array       = [Array(_o) for _o in observations.data]; 
cost_options    = prepCostOptions(obs_array, info.optimization.cost_options);

# plot the default simulations
plotTimeSeriesWithObs(out_dflt,obs_array,cost_options);
println("Outputs of plotting will be here: " * info.output.dirs.figure);



┌ Info: 
└ ----------------------------------------------


######################################################################################################


   ##    ###    #  #   ###    ####     #    ###
  #  #    #     ## #   #  #   #   #   # #   #  #
   #      #     # ##   #  #   ####   #   #  #  #
    #     #     #  #   #  #   #   #  #####  #  #
  #  #    #     #  #   #  #   #   #  #   #  #  #
   ##    ###    #  #   ###    ####   #   #  ###



######################################################################################################


┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: load configurations...
[ Info:   readConfiguration:: forcing ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/forcing.json
[ Info:   readConfiguration:: model_structure ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/model_structure.json
[ Info:   readConfiguration:: optimization ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/optimization.json
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: setup experiment...
[ Info: setupInfo: Setting and consolidating Experiment Info...
[ Info:   setExperimentBasics: setting Basic Experiment Info...
[ Info:   setExperimentOutput: setting Output Basics...
[ Info:          ...output directories set in: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4p

-------------------Forward Run Mode---------------------------



[ Info: prepTEM: preparing to run terrestrial ecosystem model (TEM)
[ Info:   prepTEMOut: preparing output and helpers...
[ Info:   helpPrepTEM: preparing helpers for running model experiment
[ Info:      preparing spatial and tem helpers
[ Info:      model run for one location and time step
[ Info:      preallocating local, threaded, and spatial data
┌ Info: 
└ ----------------------------------------------
[ Info: saving one file for all variables
[ Info: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/output_DE-Hai_WROASTED_inversion_CMAES/data/WROASTED_inversion_CMAES_DE-Hai_all_variables.zarr


118.156359 seconds (227.76 M allocations: 11.864 GiB, 13.90% gc time, 85.25% compilation time: 5% of which was recompilation)


[ Info: getExperimentInfo: load configurations...
[ Info:   readConfiguration:: forcing ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/forcing.json
[ Info:   readConfiguration:: model_structure ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/model_structure.json
[ Info:   readConfiguration:: optimization ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/optimization.json
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: setup experiment...
[ Info: setupInfo: Setting and consolidating Experiment Info...
[ Info:   setExperimentBasics: setting Basic Experiment Info...
[ Info:   setExperimentOutput: setting Output Basics...
[ Info:          ...output directories set in: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/output_DE-Hai_WROASTED_inversion_CMAES
[ Info:          ...saving a copy of json settings to: /Users/sol/dat

plot time series comparison:: gpp
plot time series comparison:: nee
plot time series comparison:: reco
plot time series comparison:: transpiration
plot time series comparison:: evapotranspiration
plot time series comparison:: agb
plot time series comparison:: ndvi
Outputs of plotting will be here: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/output_DE-Hai_WROASTED_inversion_CMAES/figure


In [19]:
# ================================== optimization ================================================= 
# run the optimization according to the settings above... can take some time...
@time out_opti  = runExperimentOpti(experiment_json; replace_info=deepcopy(replace_info), log_level=:info);

# plot the results
plotTimeSeriesWithObs(out_opti);
plotTimeSeriesDebug(out_opti.info, out_opti.output.optimized, out_opti.output.default);
println("Outputs of plotting will be here: " * info.output.dirs.figure);



┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: load configurations...
[ Info:   readConfiguration:: forcing ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/forcing.json
[ Info:   readConfiguration:: model_structure ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/model_structure.json
[ Info:   readConfiguration:: optimization ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/optimization.json
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: setup experiment...
[ Info: setupInfo: Setting and consolidating Experiment Info...
[ Info:   setExperimentBasics: setting Basic Experiment Info...
[ Info:   setExperimentOutput: setting Output Basics...
[ Info:          ...output directories set i

######################################################################################################













 .d8888b. 8888888 888b    888 8888888b.  888888b.         d8888 8888888b.
d88P  Y88b  888   8888b   888 888  "Y88b 888  "88b       d88888 888  "Y88b
Y88b.       888   88888b  888 888    888 888  .88P      d88P888 888    888
 "Y888b.    888   888Y88b 888 888    888 8888888K.     d88P 888 888    888
    "Y88b.  888   888 Y88b888 888    888 888  "Y88b   d88P  888 888    888
      "888  888   888  Y88888 888    888 888    888  d88P   888 888    888
Y88b  d88P  888   888   Y8888 888  .d88P 888   d88P d8888888888 888  .d88P
 "Y8888P" 8888888 888    Y888 8888888P"  8888888P" d88P     888 8888888P"




######################################################################################################


[ Info:         71 => cAllocationTreeFraction: Adjustment of carbon allocation according to tree cover
[ Info:         72 => autoRespiration: estimates autotrophic respiration for growth and maintenance
[ Info:         73 => cFlowSoilProperties: Effect of soil properties on the c transfers between pools
[ Info:         74 => cFlowVegProperties: Effect of vegetation properties on the c transfers between pools
[ Info:         75 => cFlow: Actual transfers of c between pools (of diagonal components)
[ Info:         76 => cCycleConsistency: Consistency checks on the c allocation and transfers between pools
[ Info:         77 => cCycle: Allocate carbon to vegetation components
[ Info:         78 => evapotranspiration: Calculate the evapotranspiration as a sum of components
[ Info:         79 => runoff: Calculate the total runoff as a sum of components
[ Info:         80 => wCycle: Apply the delta storage changes to storage variables
[ Info:         81 => waterBalance: Calculate the water ba

-------------------Optimization Mode---------------------------



[ Info:   prepTEMOut: preparing output and helpers...
┌ Info: 
└ ----------------------------------------------
[ Info: getObservation:  default_observation_data_path: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/../../data/ai4pex_2025/FLUXNET_v2023_12_1D_REPLACED_Noise003_v1.zarr
[ Info: getObservation: getting observation variables...
[ Info:  constraint: gpp
[ Info:      data: GPP_NT
[ Info:      qflag: GPP_QC_NT_merged
[ Info:      unc: ones(data)
[ Info:      weight: ones(data)
[ Info:      sel_mask: ones(data)
[ Info:      harmonize/subset...
[ Info:  
[ Info:  constraint: nee
[ Info:      data: NEE
[ Info:      qflag: NEE_QC_merged
[ Info:      unc: ones(data)
[ Info:      weight: ones(data)
[ Info:      sel_mask: ones(data)
[ Info:      harmonize/subset...
[ Info:  
[ Info:  constraint: reco
[ Info:      data: RECO_NT
[ Info:      qflag: RECO_QC_NT_merged
[ Info:      unc: ones(data)
[ Info:      weight: ones(data)
[ Info:      sel_mask: ones(data)
[ Info:    

...

[ Info: runExperimentForwardParams: forward run of the model with default/settings and input/optimized parameters...
┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: load configurations...
[ Info:   readConfiguration:: forcing ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/forcing.json
[ Info:   readConfiguration:: model_structure ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/model_structure.json
[ Info:   readConfiguration:: optimization ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_WROASTED_HB/optimization.json
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: setup experiment...
[ Info: setupInfo: Setting and consolidating Experiment Info...
[ Info:   setExperimentBasics: setting Basic Ex

######################################################################################################

 ==      ===    ==  =======  ==       ===      =======  =====       =======
 =  ====  ===  ===   ======  ==  ====  ==  ===  =====    ====  ====  ======
 =  ====  ===  ===    =====  ==  ====  ==  ====  ===  ==  ===  ====  ======
 ==  ========  ===  ==  ===  ==  ====  ==  ===  ===  ====  ==  ====  ======
 ====  ======  ===  ===  ==  ==  ====  ==      ====  ====  ==  ====  ======
 ======  ====  ===  ====  =  ==  ====  ==  ===  ===        ==  ====  ======
 =  ====  ===  ===  =====    ==  ====  ==  ====  ==  ====  ==  ====  ======
 =  ====  ===  ===  ======   ==  ====  ==  ===  ===  ====  ==  ====  ======
 ==      ===    ==  =======  ==       ===      ====  ====  ==       =======

######################################################################################################


[ Info:         71 => cAllocationTreeFraction: Adjustment of carbon allocation according to tree cover
[ Info:         72 => autoRespiration: estimates autotrophic respiration for growth and maintenance
[ Info:         73 => cFlowSoilProperties: Effect of soil properties on the c transfers between pools
[ Info:         74 => cFlowVegProperties: Effect of vegetation properties on the c transfers between pools
[ Info:         75 => cFlow: Actual transfers of c between pools (of diagonal components)
[ Info:         76 => cCycleConsistency: Consistency checks on the c allocation and transfers between pools
[ Info:         77 => cCycle: Allocate carbon to vegetation components
[ Info:         78 => evapotranspiration: Calculate the evapotranspiration as a sum of components
[ Info:         79 => runoff: Calculate the total runoff as a sum of components
[ Info:         80 => wCycle: Apply the delta storage changes to storage variables
[ Info:         81 => waterBalance: Calculate the water ba

Table with 4 columns and 7 rows:
     variable            metric     loss_opt   loss_def
   ┌───────────────────────────────────────────────────
 1 │ gpp                 NNSEInv()  0.082352   0.467601
 2 │ nee                 NNSEInv()  0.157661   0.396084
 3 │ reco                NNSEInv()  0.0861526  0.628251
 4 │ transpiration       NNSEInv()  0.0824311  0.526134
 5 │ evapotranspiration  NNSEInv()  0.0817177  0.160513
 6 │ agb                 NMAE1R()   0.664274   0.444298
 7 │ ndvi                NNSEInv()  0.598642   0.998179

1610.735528 seconds (42.98 M allocations: 6.601 GiB, 0.99% gc time, 6.67% compilation time)
plot time series comparison:: gpp
plot time series comparison:: nee
plot time series comparison:: reco
plot time series comparison:: transpiration
plot time series comparison:: evapotranspiration
plot time series comparison:: agb
plot time series comparison:: ndvi
plot debug::auto_respiration_f_airT
plot debug::k_shedding_leaf_frac
plot debug::k_shedding_root_frac
plot debug::leaf_to_reserve_frac
plot debug::reserve_to_leaf_frac
plot debug::slope_eco_stressor
plot debug::slope_eco_stressor_prev
plot debug::eco_stressor
plot debug::eco_stressor_prev
plot debug::leaf_to_reserve
plot debug::reserve_to_leaf
plot debug::reserve_to_root
plot debug::root_to_reserve
plot debug::k_shedding_root
plot debug::k_shedding_leaf
plot debug::reserve_to_root_frac
plot debug::root_to_reserve_frac
plot debug::c_eco_k_f_soilT
plot debug::c_eco_k_f_soilW
plot debug::auto_respiration
plot debug::base_runoff
plot debug

In [20]:
# ================================== another model ================================================ 
# all of the above with another model...
# only spin up the moisture pools
spinup_sequence = getSpinupSequenceSite();

# just change the model setup and experiment name
experiment_json = joinpath(@__DIR__,"settings_LUE","experiment.json");
experiment_name = "LUE_inversion_CMAES";
replace_info    = Dict("experiment.basics.time.date_begin" => "$(begin_year)-01-01",
    "experiment.basics.domain" => domain,
    "experiment.basics.name" => experiment_name,
    "experiment.basics.time.date_end" => "$(end_year)-12-31",
    "experiment.flags.run_optimization" => run_optimization,
    "experiment.model_spinup.sequence" => spinup_sequence,
    "forcing.default_forcing.data_path" => path_input,
    "forcing.subset.site" => [site_index],
    "experiment.model_output.path" => path_output,
    "optimization.observations.default_observation.data_path" => path_observation,
    );

# #=
@time out_dflt_lue  = runExperimentForward(experiment_json; replace_info=deepcopy(replace_info)); # full default model
# access some of the internals to do some plots with the forward runs...
info            = getExperimentInfo(experiment_json; replace_info=deepcopy(replace_info)); # note that this will modify information from json with the replace_info
forcing         = getForcing(info); 
run_helpers     = prepTEM(forcing, info); # not needed now
observations    = getObservation(info, forcing.helpers);
obs_array       = [Array(_o) for _o in observations.data]; 
cost_options    = prepCostOptions(obs_array, info.optimization.cost_options);
# =#

# plot the default simulations
plotTimeSeriesWithObs(out_dflt_lue,obs_array,cost_options);
println("Outputs of plotting will be here: " * info.output.dirs.figure);

# run the optimization
@time out_lue_opti  = runExperimentOpti(experiment_json; replace_info=deepcopy(replace_info), log_level=:info);

# plot the results
plotTimeSeriesWithObs(out_lue_opti);
println("Outputs of plotting will be here: " * out_lue_opti.info.output.dirs.figure);



######################################################################################################

  ______   ______  __    __  _______   _______    ______   _______
 /      \ |      \|  \  |  \|       \ |       \  /      \ |       \
|  $$$$$$\ \$$$$$$| $$\ | $$| $$$$$$$\| $$$$$$$\|  $$$$$$\| $$$$$$$\
| $$___\$$  | $$  | $$$\| $$| $$  | $$| $$__/ $$| $$__| $$| $$  | $$
 \$$    \   | $$  | $$$$\ $$| $$  | $$| $$    $$| $$    $$| $$  | $$
 _\$$$$$$\  | $$  | $$\$$ $$| $$  | $$| $$$$$$$\| $$$$$$$$| $$  | $$
|  \__| $$ _| $$_ | $$ \$$$$| $$__/ $$| $$__/ $$| $$  | $$| $$__/ $$
 \$$    $$|   $$ \| $$  \$$$| $$    $$| $$    $$| $$  | $$| $$    $$
  \$$$$$$  \$$$$$$ \$$   \$$ \$$$$$$$  \$$$$$$$  \$$   \$$ \$$$$$$$




######################################################################################################


┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: load configurations...
[ Info:   readConfiguration:: forcing ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_LUE/forcing_zarr.json
[ Info:   readConfiguration:: model_structure ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_LUE/model_structure_LUE.json
[ Info:   readConfiguration:: optimization ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_LUE/optimization.json
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: setup experiment...
[ Info: setupInfo: Setting and consolidating Experiment Info...
[ Info:   setExperimentBasics: setting Basic Experiment Info...
[ Info:   setExperimentOutput: setting Output Basics...
[ Info:          ...output directories set in: /Users/sol/d

-------------------Forward Run Mode---------------------------



[ Info: prepTEM: preparing to run terrestrial ecosystem model (TEM)
[ Info:   prepTEMOut: preparing output and helpers...
[ Info:   helpPrepTEM: preparing helpers for running model experiment
[ Info:      preparing spatial and tem helpers
[ Info:      model run for one location and time step
[ Info:      preallocating local, threaded, and spatial data
┌ Info: 
└ ----------------------------------------------
[ Info: saving one file for all variables
[ Info: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/output_DE-Hai_LUE_inversion_CMAES/data/LUE_inversion_CMAES_DE-Hai_all_variables.zarr


 14.807102 seconds (15.79 M allocations: 1.228 GiB, 12.58% gc time, 86.33% compilation time: <1% of which was recompilation)


...

[ Info: runExperimentForwardParams: forward run of the model with default/settings and input/optimized parameters...
┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: load configurations...
[ Info:   readConfiguration:: forcing ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_LUE/forcing_zarr.json
[ Info:   readConfiguration:: model_structure ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_LUE/model_structure_LUE.json
[ Info:   readConfiguration:: optimization ::: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/settings_LUE/optimization.json
┌ Info: 
└ ----------------------------------------------
[ Info: getExperimentInfo: setup experiment...
[ Info: setupInfo: Setting and consolidating Experiment Info...
[ Info:   setExperimentBasics: setting Basic Experiment Info..

######################################################################################################


















      ,gg,         ,a8a,  ,ggg, ,ggggggg,   ,gggggggggggg,    ,ggggggggggg,             ,ggg,  ,gggggggggggg,
     i8""8i       ,8" "8,dP""Y8,8P"""""Y8b dP"""88""""""Y8b, dP"""88""""""Y8,          dP""8I dP"""88""""""Y8b,
     `8,,8'       d8   8bYb, `8dP'     `88 Yb,  88       `8b,Yb,  88      `8b         dP   88 Yb,  88       `8b,
      `88'        88   88 `"  88'       88  `"  88        `8b `"  88      ,8P        dP    88  `"  88        `8b
      dP"8,       88   88     88        88      88         Y8     88aaaad8P"        ,8'    88      88         Y8
     dP' `8a      Y8   8P     88        88      88         d8     88""""Y8ba        d88888888      88         d8
    dP'   `Yb     `8, ,8'     88        88      88        ,8P     88      `8b __   ,8"     88      88        ,8P
_ ,dP'     I88888  "8,8"      88        88      88       ,8P'     88      ,8PdP"  ,8P      Y

[ Info:         71 => cAllocationTreeFraction: Adjustment of carbon allocation according to tree cover
[ Info:         72 => autoRespiration: estimates autotrophic respiration for growth and maintenance
[ Info:         73 => cFlowSoilProperties: Effect of soil properties on the c transfers between pools
[ Info:         74 => cFlowVegProperties: Effect of vegetation properties on the c transfers between pools
[ Info:         75 => cFlow: Actual transfers of c between pools (of diagonal components)
[ Info:         76 => cCycleConsistency: Consistency checks on the c allocation and transfers between pools
[ Info:         77 => cCycle: Allocate carbon to vegetation components
[ Info:         78 => evapotranspiration: Calculate the evapotranspiration as a sum of components
[ Info:         79 => runoff: Calculate the total runoff as a sum of components
[ Info:         80 => wCycle: Apply the delta storage changes to storage variables
[ Info:         81 => waterBalance: Calculate the water ba

Table with 4 columns and 1 row:
     variable  metric     loss_opt  loss_def
   ┌────────────────────────────────────────
 1 │ gpp       NNSEInv()  0.10339   0.603511

 33.267019 seconds (12.25 M allocations: 1.865 GiB, 5.82% gc time, 81.33% compilation time)
plot time series comparison:: gpp
Outputs of plotting will be here: /Users/sol/data/go/hub/SINDBAD-Tutorials/tutorials/ai4pex_2025/output_DE-Hai_LUE_inversion_CMAES/figure


In [21]:
# ================================== time for discussion ========================================== 